In [4]:
import scipy as sp
import numpy as np

In [14]:
A = np.diag([10, 2, 3, 4, 5, 6])
b = np.array([2, 3, 4, 5, 6, 7])
M = 10

def cost_function(x):
    if not isinstance(x, np.ndarray):
        raise TypeError
    return 1/2 * (x.T @ A @ x) + b.T @ x

c1 = np.array([1, 1, 0, 0, 0, 0])
c2 = np.array([-1, 0, 1, 0, 0, 1])
c3 = np.array([0, -1, 0, 1, 1, 0])

def constraint_1(x):
    return c1 @ x - M

def constraint_2(x):
    return c2 @ x

def constraint_3(x):
    return c3 @ x

bounds = [(0, None) for _ in range(6)]
constraints = [
    {"type": "eq", "fun": constraint_1},
    {"type": "eq", "fun": constraint_2},
    {"type": "eq", "fun": constraint_3},
]

x0 = np.ones(6)
result = sp.optimize.minimize(cost_function, x0, method='SLSQP', constraints=constraints, bounds=bounds)
if result.success:
    print("Optimal solution:", result.x)
    print("Objective value:", result.fun)
else:
    print("Optimization failed:", result.message)

Optimal solution: [2.69177945 7.30822055 2.12785608 4.17123605 3.1369845  0.56392337]
Objective value: 236.2294520548796


In [15]:
x = result.x
x

array([2.69177945, 7.30822055, 2.12785608, 4.17123605, 3.1369845 ,
       0.56392337])

In [ ]:
r1 = np.array([1, 0, 0, 0, 0, 1])
r2 = np.array([1, 0, 1, 0, 0, 0])
r3 = np.array([0, 1, 0, 1, 0, 0])
r4 = np.array([0, 1, 0, 0, 1, 0])
def route(x, r):
    return r.T @ A @ x + r.T @ b

(np.float64(39.30133467947831),
 np.float64(39.301362696883416),
 np.float64(39.301385313685834),
 np.float64(39.301363616341895))